# Minimum EigenValue For 4x4 Matrix

In [15]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import pennylane as qml
import numpy as np
from numpy import pi
simulator = Aer.get_backend('qasm_simulator')



# Matrix and Decomposition

In [16]:
M= np.array([[-1. ,  0. ,  0. ,  0. ],
       [ 0. ,  1.5,  0.5, -0.5],
       [ 0. ,  0.5 ,  2. ,  0.5 ],
       [ 0. , -0.5,  0.5,  1.5]])

In [17]:
coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
H = qml.Hamiltonian(coeffs, obs_list)

In [18]:
print(H)

(1.0) [I0 I1]
+ (0.25) [I0 X1]
+ (-0.5) [I0 Z1]
+ (-0.25) [X0 I1]
+ (0.25) [X0 X1]
+ (0.25) [X0 Z1]
+ (0.25) [Y0 Y1]
+ (-0.75) [Z0 I1]
+ (-0.25) [Z0 X1]
+ (-0.75) [Z0 Z1]


In [94]:
def coef(H):
    coeff = [0]*16
    h = str(H)
    for i in range (len(h)):
        p = ""
        if h[i] =="(":
            j = i+1 
            while h[j]!=")" :
                p=p+h[j]
                j+=1
            
            if h[j+3] == "I" and h[j+6] == "I":
                coeff[0] = float(p)
                
            if h[j+3] == "Z" and h[j+6] == "Z":
                coeff[1] = float(p)
                
            if h[j+3] == "X" and h[j+6] == "X":
                coeff[2] = float(p)
                
            if h[j+3] == "Y" and h[j+6] == "Y":
                coeff[3] = float(p)
            
            if h[j+3] == "I" and h[j+6] == "X":
                coeff[4] = float(p)
                
            if h[j+3] == "X" and h[j+6] == "I":
                coeff[5] = float(p)
                
            if h[j+3] == "I" and h[j+6] == "Y":
                coeff[6] = float(p)
                
            if h[j+3] == "Y" and h[j+6] == "I":
                coeff[7] = float(p)
                
            if h[j+3] == "I" and h[j+6] == "Z":
                coeff[8] = float(p)
                
            if h[j+3] == "Z" and h[j+6] == "I":
                coeff[9] = float(p)
                
            if h[j+3] == "X" and h[j+6] == "Y":
                coeff[10] = float(p)
                
            if h[j+3] == "X" and h[j+6] == "Y":
                coeff[11] = float(p)
                
            if h[j+3] == "x" and h[j+6] == "z":
                coeff[12] = float(p)
                
            if h[j+3] == "z" and h[j+6] == "x":
                coeff[13] = float(p)
                
            if h[j+3] == "y" and h[j+6] == "z":
                coeff[14] = float(p)
                
            if h[j+3] == "z" and h[j+6] == "y":
                coeff[15] = float(p)
                
    return(coeff)
            
            

# Ansatz

In [95]:
def Operational_circuit(params):
    qc = QuantumCircuit(2)
    
    #qc.h(range(2))
    qc.rx(params[0],0)
    qc.rx(params[1],1)
    qc.ry(params[2],0)
    qc.ry(params[3],1)
    qc.cx(0,1)
    qc.ry(params[4],0)
    qc.ry(params[5],1)
    
    return qc
    

In [96]:
Operational_circuit_try = Operational_circuit([1,2,3,4,5,6])

In [97]:
Operational_circuit_try.draw()

┌───────┐┌───────┐     ┌───────┐
q_0: ┤ RX(1) ├┤ RY(3) ├──■──┤ RY(5) ├
     ├───────┤├───────┤┌─┴─┐├───────┤
q_1: ┤ RX(2) ├┤ RY(4) ├┤ X ├┤ RY(6) ├
     └───────┘└───────┘└───┘└───────┘

# Measurments for different components of H

# 1. ZZ

In [98]:
def measure_zz_circuit(given_circuit):
    zz_meas = given_circuit.copy()
    zz_meas.measure_all()
    return zz_meas

zz_meas = measure_zz_circuit(Operational_circuit_try)
zz_meas.draw()

┌───────┐┌───────┐     ┌───────┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├──■──┤ RY(5) ├─░─┤M├───
        ├───────┤├───────┤┌─┴─┐├───────┤ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ X ├┤ RY(6) ├─░──╫─┤M├
        └───────┘└───────┘└───┘└───────┘ ░  ║ └╥┘
meas_0: ════════════════════════════════════╩══╬═
                                               ║ 
meas_1: ═══════════════════════════════════════╩═

In [99]:
def measure_zz(given_circuit, num_shots = 10000):

    zz_meas = measure_zz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

In [100]:
zz = measure_zz(Operational_circuit_try)
print("<ZZ> =", str(zz))

<ZZ> = -0.6212


# 2. XX

In [101]:
def measure_xx_circuit(given_circuit):
    xx_meas = given_circuit.copy()
    xx_meas.h(0)
    xx_meas.h(1)
    xx_meas.measure_all()

    return xx_meas

In [102]:
xx_meas = measure_xx_circuit(Operational_circuit_try)
xx_meas.draw()

┌───────┐┌───────┐     ┌───────┐┌───┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├──■──┤ RY(5) ├┤ H ├─░─┤M├───
        ├───────┤├───────┤┌─┴─┐├───────┤├───┤ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ X ├┤ RY(6) ├┤ H ├─░──╫─┤M├
        └───────┘└───────┘└───┘└───────┘└───┘ ░  ║ └╥┘
meas_0: ═════════════════════════════════════════╩══╬═
                                                    ║ 
meas_1: ════════════════════════════════════════════╩═

In [103]:
def measure_xx(given_circuit, num_shots = 10000):
    
    xx_meas = measure_xx_circuit(given_circuit)
    
    result = execute(xx_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(xx_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    xx = counts['00'] + counts['11'] - counts['01'] - counts['10']
    xx = xx / total_counts
    
    return xx

In [104]:
xx = measure_xx(Operational_circuit_try)
print("<XX> =", str(xx))

<XX> = 0.3066


# 3. YY

In [105]:
def measure_yy_circuit(given_circuit):
    yy_meas = given_circuit.copy()
    yy_meas.rz(-pi/2,0)
    yy_meas.rz(-pi/2,1)

    yy_meas.h(0)
    yy_meas.h(1)
    yy_meas.measure_all()

    return yy_meas

In [106]:
yy_meas = measure_yy_circuit(Operational_circuit_try)
yy_meas.draw()

┌───────┐┌───────┐     ┌───────┐┌───────────┐┌───┐ ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├──■──┤ RY(5) ├┤ RZ(-pi/2) ├┤ H ├─░─┤M├───
        ├───────┤├───────┤┌─┴─┐├───────┤├───────────┤├───┤ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ X ├┤ RY(6) ├┤ RZ(-pi/2) ├┤ H ├─░──╫─┤M├
        └───────┘└───────┘└───┘└───────┘└───────────┘└───┘ ░  ║ └╥┘
meas_0: ══════════════════════════════════════════════════════╩══╬═
                                                                 ║ 
meas_1: ═════════════════════════════════════════════════════════╩═

In [107]:
def measure_yy(given_circuit, num_shots = 10000):
    
    yy_meas = measure_yy_circuit(given_circuit)
    
    result = execute(yy_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(yy_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    yy = counts['00'] + counts['11'] - counts['01'] - counts['10']
    yy = yy / total_counts
    
    return yy

In [108]:
yy = measure_yy(Operational_circuit_try)
print("<yy> =", str(yy))

<yy> = -0.03


# 4. IX

In [109]:
def measure_ix_circuit(given_circuit):
    ix_meas = given_circuit.copy()
    ix_meas.h(1)
    ix_meas.measure_all()

    return ix_meas
def measure_ix(given_circuit, num_shots = 10000):
    
    ix_meas = measure_ix_circuit(given_circuit)
    
    result = execute(ix_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(ix_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    ix = counts['00'] - counts['11'] + counts['01'] - counts['10']
    ix = ix / total_counts
    
    return ix

In [110]:
ix_meas = measure_ix_circuit(Operational_circuit_try)
ix_meas.draw()

┌───────┐┌───────┐     ┌───────┐      ░ ┌─┐   
   q_0: ┤ RX(1) ├┤ RY(3) ├──■──┤ RY(5) ├──────░─┤M├───
        ├───────┤├───────┤┌─┴─┐├───────┤┌───┐ ░ └╥┘┌─┐
   q_1: ┤ RX(2) ├┤ RY(4) ├┤ X ├┤ RY(6) ├┤ H ├─░──╫─┤M├
        └───────┘└───────┘└───┘└───────┘└───┘ ░  ║ └╥┘
meas_0: ═════════════════════════════════════════╩══╬═
                                                    ║ 
meas_1: ════════════════════════════════════════════╩═

In [111]:
ix = measure_ix(Operational_circuit_try)
print("<IX> =", str(ix))

<IX> = 0.3262


# 5. XI

In [112]:
def measure_xi_circuit(given_circuit):
    xi_meas = given_circuit.copy()
    xi_meas.h(0)
    xi_meas.measure_all()

    return xi_meas
def measure_xi(given_circuit, num_shots = 10000):
    
    xi_meas = measure_xi_circuit(given_circuit)
    
    result = execute(xi_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(xi_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    xi = counts['00'] - counts['11'] - counts['01'] + counts['10']
    xi = xi / total_counts
    
    return xi
xi = measure_xi(Operational_circuit_try)
print("<XI> =", str(xi))

<XI> = 0.5174


# 6. IY

In [113]:
def measure_iy_circuit(given_circuit):
    iy_meas = given_circuit.copy()
    iy_meas.rz(-pi/2,1)
    iy_meas.h(1)
    iy_meas.measure_all()

    return iy_meas
def measure_iy(given_circuit, num_shots = 10000):
    
    iy_meas = measure_iy_circuit(given_circuit)
    
    result = execute(iy_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(iy_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    iy = counts['00'] - counts['11'] + counts['01'] - counts['10']
    iy = iy / total_counts
    
    return iy

# 7. YI

In [114]:
def measure_yi_circuit(given_circuit):
    yi_meas = given_circuit.copy()
    yi_meas.rz(-pi/2,0)
    yi_meas.h(0)
    yi_meas.measure_all()

    return yi_meas
def measure_yi(given_circuit, num_shots = 10000):
    
    yi_meas = measure_yi_circuit(given_circuit)
    
    result = execute(yi_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(yi_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    yi = counts['00'] - counts['11'] - counts['01'] + counts['10']
    yi = yi / total_counts
    
    return yi

# 8. IZ

In [115]:
def measure_iz(given_circuit, num_shots = 10000):

    zz_meas = measure_zz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] - counts['11']+ counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

# 9. ZI

In [116]:
def measure_zi(given_circuit, num_shots = 10000):

    zz_meas = measure_zz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] - counts['11']- counts['01'] + counts['10']
    zz = zz / total_counts
    
    return zz

# 10. XY

In [117]:
def measure_xy_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.rz(-pi/2,1)
    xy_meas.h(1)
    xy_meas.h(0)
    xy_meas.measure_all()

    return xy_meas

def measure_xy(given_circuit, num_shots = 10000):

    zz_meas = measure_xy_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

# 11. YX

In [118]:
def measure_yx_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.rz(-pi/2,0)
    xy_meas.h(0)
    xy_meas.h(1)
    xy_meas.measure_all()

    return xy_meas

def measure_yx(given_circuit, num_shots = 10000):

    zz_meas = measure_yx_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

# 12. XZ

In [119]:
def measure_xz_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.h(0)
    xy_meas.measure_all()

    return xy_meas

def measure_xz(given_circuit, num_shots = 10000):

    zz_meas = measure_xz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

# 13. ZX

In [120]:
def measure_zx_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.h(1)
    xy_meas.measure_all()

    return xy_meas

def measure_zx(given_circuit, num_shots = 10000):

    zz_meas = measure_zx_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

# 14. YZ

In [121]:
def measure_yz_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.rz(-pi/2,0)
    xy_meas.h(0)
    xy_meas.measure_all()

    return xy_meas

def measure_yz(given_circuit, num_shots = 10000):

    zz_meas = measure_yz_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

# 15. ZY

In [122]:
def measure_zy_circuit(given_circuit):
    xy_meas = given_circuit.copy()
    xy_meas.rz(-pi/2,1)
    xy_meas.h(1)
    xy_meas.measure_all()

    return xy_meas

def measure_zy(given_circuit, num_shots = 10000):

    zz_meas = measure_zy_circuit(given_circuit)
    
    result = execute(zz_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(zz_meas)

    if '00' not in counts:
        counts['00'] = 0
    if '01' not in counts:
        counts['01'] = 0
    if '10' not in counts:
        counts['10'] = 0
    if '11' not in counts:
        counts['11'] = 0 

    total_counts = counts['00'] + counts['11'] + counts['01'] + counts['10']
    zz = counts['00'] + counts['11'] - counts['01'] - counts['10']
    zz = zz / total_counts
    
    return zz

# Cost_Function

In [133]:
def eigen_value(params):
    num_shots = 100000
    zz = measure_zz(Operational_circuit(params), num_shots = num_shots)
    xx = measure_xx(Operational_circuit(params), num_shots = num_shots)
    yy = measure_yy(Operational_circuit(params), num_shots = num_shots)
    ix = measure_ix(Operational_circuit(params), num_shots = num_shots)
    xi = measure_xi(Operational_circuit(params), num_shots = num_shots)
    iy = measure_iy(Operational_circuit(params), num_shots = num_shots)
    yi = measure_yi(Operational_circuit(params), num_shots = num_shots)
    iz = measure_iz(Operational_circuit(params), num_shots = num_shots)
    zi = measure_zi(Operational_circuit(params), num_shots = num_shots)
    xy = measure_xy(Operational_circuit(params), num_shots = num_shots)
    yx = measure_yx(Operational_circuit(params), num_shots = num_shots)
    xz = measure_xz(Operational_circuit(params), num_shots = num_shots)
    zx = measure_zx(Operational_circuit(params), num_shots = num_shots)
    yz = measure_yz(Operational_circuit(params), num_shots = num_shots)
    zy = measure_zy(Operational_circuit(params), num_shots = num_shots)
    
    coeff = coef(H)
    
    ev = (coeff[0]*1 + coeff[1]*zz + coeff[2]*xx+coeff[3]*yy+coeff[4]*ix+coeff[5]*xi+coeff[6]*iy+coeff[7]*yi+coeff[8]*iz+
         coeff[9]*zi+coeff[10]*xy+coeff[11]*yx+coeff[12]*xz+coeff[13]*zx+coeff[14]*yz+coeff[15]*zy)
    
    return ev

In [134]:
params = [0,0,0,0,0,0]

In [135]:
ev = eigen_value(params)     #Try energy with params = [0,0,0,0,0,0]
print("The Eigen_Value of the trial state is", str(ev))

The Eigen_Value of the trial state is -0.9989049999999999


# Finding Lowest Eignevalue

In [136]:
from qiskit.aqua.components.optimizers import COBYLA,SLSQP,SPSA

optimizer = COBYLA(50,0.001)

In [137]:
ret = optimizer.optimize(num_vars=6, objective_function=eigen_value, initial_point=params)

In [138]:
eigen_value(ret[0])

-1.046875

In [139]:
ret[0]

array([-6.67246802e-02, -4.79907340e-02, -5.59187975e-02, -2.14918315e-04,
        2.61880395e-01, -2.52326013e-01])

In [ ]:
backend_state = Aer.get_backend('statevector_simulator')
result = execute(Operational_circuit(ret[0]),backend_state).result()
out_state = result.get_statevector()
out_state